# Section 1

## Installing and collecting data from Wikipedia

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 17.7MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 37.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [91]:
#import modules
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Get data using BeautifulSoup and import into Pandas

In [31]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wikisource = requests.get(url).text
soup = BeautifulSoup(wikisource, 'lxml')
data = soup.find('table')

In [33]:
#Make a dataframe using data source
postcode = []
borough = []
neighborhood = []

for tr in data.find_all('tr'):
    i = 1
    for td in tr.find_all('td'):
        if i == 1:
            postcode.append(td.text)
        elif i == 2:
            borough.append(td.text)
        elif i == 3:
            neighborhood.append(td.text.replace('\n',''))
        i += 1
        
headers = {'PostalCode': postcode, 'Borough': borough, 'Neighborhood': neighborhood}

df = pd.DataFrame.from_dict(headers)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Preprocessing

In [34]:
# Remove "not assigned"
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [38]:
# Grouping
boroughlist = df.groupby(['PostalCode'])['Borough'].apply(set)
neighborhoodlist = df.groupby(['PostalCode'])['Neighborhood'].apply(set)
df_list = pd.merge(pd.DataFrame(boroughlist), pd.DataFrame(neighborhoodlist), how='outer', on=['PostalCode'])
df_list.reset_index(inplace=True)
for i in range(len(df_list)-1):
    df_list.Neighborhood[i] = ', '.join(df_list.Neighborhood[i])
    df_list.Borough[i] = ', '.join(df_list.Borough[i])

In [39]:
#Replacing "not assigned" neighborhoods with borough name
def rename(Neighborhood, Borough):
    if Neighborhood =='Not assigned':
        return Borough
    else:
        return Neighborhood
df_list.Neighborhood = df_list.apply(lambda row: rename(row.Neighborhood, row.Borough), axis = 1)
df_list.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [40]:
df_list[df_list.PostalCode == "M7A"]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [41]:
df_list[df_list.PostalCode == "M5A"]

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [42]:
#Shape of dataframe
df_list.shape

(103, 3)

# Section 2

## Utilizing CSV Doc to merge geodata with dataframe

In [48]:
df_geocodes = pd.read_csv('http://cocl.us/Geospatial_data')
df_geocodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
#adjusting column names so dataframes can merge
df_geocodes.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_geocodes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
print(df_geocodes.shape)
print(df_list.shape)

(103, 3)
(103, 3)


In [53]:
#Merging
df_geolist = pd.merge(df_list, df_geocodes, how='outer', on=['PostalCode'])
df_geolist.reset_index(inplace=True)
df_geolist = df_geolist[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
df_geolist

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill...",43.688905,-79.554724
101,M9V,Etobicoke,"Mount Olive, South Steeles, Thistletown, Silve...",43.739416,-79.588437


# Section 3

## Filter to only Toronto boroughs

In [120]:
df_toronto = df_geolist[df_geolist.apply(lambda row: 'Toronto' in row.Borough, axis = 1)]
df_toronto.reset_index(inplace=True)
df_toronto = df_toronto[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Forest Hill SE, Deer Park, So...",43.686412,-79.400049


## Mapping Toronto

In [56]:
import folium
from geopy.geocoders import Nominatim

In [57]:
address = "Toronto, Ontario, Canada"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario, Canada are 43.653963, -79.387207.


In [63]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Foursquare credentials

In [65]:
import requests 
from pandas.io.json import json_normalize 

In [64]:
CLIENT_ID = 'CXKBFOT0NES1BWKP2254OD1RN1I2OSKZSV35NAK2V2OUJRIS' # your Foursquare ID
CLIENT_SECRET = 'YPICYNAIJBOVYXZTXNA2UB3K2Z1T1CR2NHDMM1NHHB32TURO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CXKBFOT0NES1BWKP2254OD1RN1I2OSKZSV35NAK2V2OUJRIS
CLIENT_SECRET:YPICYNAIJBOVYXZTXNA2UB3K2Z1T1CR2NHDMM1NHHB32TURO


## Exploring neighborhoods in Toronto

In [70]:
# Defining radius and limit
LIMIT = 100
radius = 50

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Identifying Toronto venues

In [75]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )



The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Summerhill West, Forest Hill SE, Deer Park, South Hill, Rathnelly
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, Richmond, King
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill West, Forest Hill North
Yorkville, The Annex, North Midtown
University of Toronto, Harbord
Kensington Market, Grange Park, Chinatown
South Niagara, Bathurst Quay, King and Spadina, Railway Lands, CN Tower, Harbourfront West, Island airport
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Exhibition Place, Parkdale Village, Brockton
High Park, The 

In [76]:
print(toronto_venues.shape)
toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors


## Number of values per neighborhood

In [77]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
Central Bay Street,86,86,86,86,86,86
Christie,16,16,16,16,16,16
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,7,7,7,7,7,7


## Unique categories

In [78]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyzing neighborhoods

In [80]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
toronto_onehot.shape

(1712, 238)

In [83]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, Richmond, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.066667
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.00,...,0.000000,0.000000,0.00,0.011628,0.000000,0.000000,0.011628,0.0000,0.000000,0.011628
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
5,Church and Wellesley,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.00,...,0.000000,0.000000,0.00,0.000000,0.011628,0.011628,0.000000,0.0000,0.011628,0.011628
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.00,...,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,...,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000


## Identifying top venues

In [84]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  American Restaurant  0.04
3                  Bar  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.07
1  Light Rail Station  0.07
2                Park  0.07
3          Comic Shop  0.07
4    Recording Studio  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.03
4      Ice Cream Shop  0.03


----Christie----
                venue  freq
0       Grocery Store  0.19
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4         Coffee Sh

In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Hotel,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Beer Bar,Steakhouse,Seafood Restaurant,Bakery,Irish Pub
2,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Farmers Market,Fast Food Restaurant
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Gym / Fitness Center,Chinese Restaurant,Bar
4,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Restaurant,Bank,Italian Restaurant,Diner,Nightclub


## Clustered Neighborhoods

In [116]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [126]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [127]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, Richmond, King",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Hotel,Asian Restaurant,Burger Joint
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Beer Bar,Steakhouse,Seafood Restaurant,Bakery,Irish Pub
2,0,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Farmers Market,Fast Food Restaurant
3,0,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Gym / Fitness Center,Chinese Restaurant,Bar
4,0,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Restaurant,Bank,Italian Restaurant,Diner,Nightclub


In [130]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

In [135]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Park,Other Great Outdoors,Neighborhood,Trail,Pub,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Brewery,Bubble Tea Shop,Café,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pizza Place,Pet Store,Movie Theater,Brewery,Burger Joint,Burrito Place,Pub,Park,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Brewery,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Photography Studio,Park,Bus Line,Swim School,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Park,Other Great Outdoors,Neighborhood,Trail,Pub,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Brewery,Bubble Tea Shop,Café,Restaurant
2,East Toronto,0,Sandwich Place,Pizza Place,Pet Store,Movie Theater,Brewery,Burger Joint,Burrito Place,Pub,Park,Fast Food Restaurant
3,East Toronto,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Brewery,Seafood Restaurant,Sandwich Place
5,Central Toronto,0,Food & Drink Shop,Clothing Store,Breakfast Spot,Park,Gym,Sandwich Place,Hotel,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Burger Joint,Salon / Barbershop,Café,Restaurant,Rental Car Location,Chinese Restaurant,Clothing Store,Yoga Studio
7,Central Toronto,0,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Gym,Sushi Restaurant,Coffee Shop,Pizza Place,Fried Chicken Joint,Deli / Bodega
9,Central Toronto,0,Pub,Coffee Shop,Bagel Shop,Light Rail Station,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Liquor Store
11,Downtown Toronto,0,Coffee Shop,Park,Café,Restaurant,Italian Restaurant,Pub,Bakery,Pizza Place,Sandwich Place,Butcher
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pizza Place,Bubble Tea Shop,Burger Joint,Hotel,Gym


In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Home Service,Garden,Yoga Studio,Doner Restaurant,Filipino Restaurant,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Trail,Jewelry Store,Park,Sushi Restaurant,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio


In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Photography Studio,Park,Bus Line,Swim School,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Playground,Park,Restaurant,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
10,Downtown Toronto,4,Park,Trail,Building,Playground,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio


# Fin